In [ ]:
from sympy.physics.quantum.cg import CG
from itertools import product
import pandas as pd

In [ ]:
allowed_channels = {
    "l_max": 4,
    "s_max": 1,
    "j_max": 2,
}

In [ ]:
def m_range(j):
    return range(-j, j + 1)


def get_two_n_channels(l_max, j_max, s_max=1, numeric=False):
    channels = []

    for j, s, l in product(*map(range, [j_max + 1, s_max + 1, l_max + 1])):

        if not (abs(l - s) <= j <= l + s):
            continue

        for mj, ms, ml in product(*map(m_range, [j, s, l])):
            if ml + ms != mj:
                continue

            if ml == ms == mj == 0 and (l + s - j) % 2 == 1:
                # if all m signs flip, the CG is the same up to (-1)**(l+s-j)
                # thus if all m==0, l+s-j must be even
                continue

            cg = CG(l, ml, s, ms, j, mj).doit()
            yield {
                "l": l,
                "ml": ml,
                "s": s,
                "ms": ms,
                "j": j,
                "mj": mj,
                "cg": cg.evalf() if numeric else cg,
            }


channels = list(get_two_n_channels(**allowed_channels, numeric=False))

df = pd.DataFrame(channels)
df.head()

In [ ]:
transitions = [
    lambda i, o: i["s"] == o["s"] and i["ms"] == o["ms"] and i["mj"] == o["mj"]
]
transitions = []

In [ ]:
def get_matrix_channels(two_n_channels, allowed_transitions):
    channels = []
    for c_i, c_o in product(two_n_channels.copy(), two_n_channels.copy()):
        if not allowed_transitions or any([allowed(c_i, c_o) for allowed in allowed_transitions]):
            data = {"cg": c_i["cg"] * c_o["cg"]}
            data.update({key + "_i": val for key, val in c_i.items()})
            data.update({key + "_o": val for key, val in c_o.items()})
            yield data


df = pd.DataFrame(get_matrix_channels(channels, transitions))
df.head()

In [ ]:
df["mla"] = df["ml_i"] - df["ml_o"]
cols = ["s_o", "ms_o", "s_i", "ms_i", "mla"]
pwd_chan = df[cols].drop_duplicates().sort_values(cols,)
pwd_chan